<a href="https://colab.research.google.com/github/schatz06/EPP445_Project/blob/main/eye_landamarks_recognition_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary packages

In [6]:
import cv2
import dlib
import os
import re

In [ ]:
!wget http://dlib.net/files/data/ibug_300W_large_face_landmark_dataset.tar.gz

In [ ]:
!tar xvzf ibug_300W_large_face_landmark_dataset.tar.gz

Regex

In [7]:
REG_PART = re.compile("part name='[0-9]+'")
REG_NUM = re.compile("[0-9]+")

Function that parse the xml file of the train data to get the parts of the face that we want to train our predictor

In [2]:
def parse_xml(input_path,output_path, parts):
  file = open(input_path,"r") # Open the xml file with the training data
  out_file = open(output_path,"w") # Open a file descriptor to write the parsed xml file
  point_set = set(parts) #create a set with all the parts I want to traing my model

  for line in file.readlines():
    finds = re.findall(REG_PART,line)

    #Trying to find the part section we want
    if len(finds) <= 0:
      out_file.write(line)
    else:
      #Here we found a part section
      #So we take the part's name and the x,y coordinates
      name, x, y = re.findall(REG_NUM,line)

      #Here we check if a part's name match with what we are looking for and then write to output file
      if int(name) in point_set:
        out_file.write(f"      <part name='{name}' x='{x}' y='{y}'/>\n")

  out_file.close()


THIS PARAMETER determines which parts of the face we want to choose for the training, In our example is the left and right eye

In [8]:
EYES = [i for i in range(36,48)]

This method set the options for our model and trains the model

In [9]:
def train_model(name,xml):
  # Here we configure the training options
  options = dlib.shape_predictor_training_options()
  options.tree_depth = 3
  options.nu = 0.1
  options.cascade_depth = 10
  options.feature_pool_size = 150
  options.num_test_splits = 350
  options.oversampling_amount = 5
  options.oversampling_translation_jitter = 0
  options.be_verbose = True
  options.num_threads = 1

  dlib.train_shape_predictor(xml, name, options)



This method measures the error of the model on the given xml file

In [ ]:
def measure_model_error(model,xml_annotations):
  error = dlib.test_shape_predictor(xml_annotations, model)
  print("Error of the model: {} is {}".format(model, error))

In [3]:
if __name__ == '__main__':
  ibug_xml_train = "ibug_300W_large_face_landmark_dataset/labels_ibug_300W_train.xml"
  ibug_xml_test = "ibug_300W_large_face_landmark_dataset/labels_ibug_300W_test.xml"
  eyes_xml_train = "ibug_300W_large_face_landmark_dataset/eyes_train.xml"
  eyes_xml_test = "ibug_300W_large_face_landmark_dataset/eyes_test.xml"
  eyes_dat = "eyes.dat"

  #parse the TRAIN xml and create the new one with only the eyes annotations
  parse_xml(ibug_xml_train, eyes_xml_train, parts = EYES)
  print("Hello1")
  
  #parse the TEST xml and create the new one with only the eyes annotations
  parse_xml(ibug_xml_test, eyes_xml_test, parts = EYES)
  print("Hello2")

  #Here we train the eye model
  train_model(eyes_dat, eyes_xml_train)

  #And last we measure the training and test error
  measure_model_error(eyes_dat, eyes_xml_train)
  measure_model_error(eyes_dat, eyes_xml_test)


NameError: ignored

Here we define the landmark point for the eyes.

*   68 point in a face
*   22 points for eyes and eyebrows






Dataset used iBug 300W


*   Download
*   Extract






Paths 